# Multimodal Click-Through Rate Prediction (MM-CTR)

This notebook presents an end-to-end multimodal CTR prediction system combining
user behavior sequences, categorical interaction features, and precomputed
multimodal item embeddings.


## 1. Introduction

Click-Through Rate (CTR) prediction is a core problem in recommender systems,
aiming to estimate the probability that a user will interact with a given item.
Accurate CTR prediction enables personalized content ranking, advertising
optimization, and improved user engagement.

In this project, we address multimodal CTR prediction by integrating:
- User–item interaction histories,
- Behavioral feedback signals (likes and views),
- Sequential browsing patterns,
- Precomputed multimodal item embeddings.

The objective is to design, train, and evaluate a deep learning model capable of
capturing both sequential user behavior and rich item representations within a
single, reproducible pipeline.


In [1]:
# Force a compatible protobuf version
# Some libraries (CLIP, sentence-transformers) may break with newer versions
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; pyth

In [1]:
# Install OpenAI CLIP directly from GitHub
# Kaggle does not provide it by default
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-cbl9alrk
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-cbl9alrk
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 

In [2]:
# Additional libraries for training & evaluation
!pip install -q polars tqdm

In [3]:
# ==========================
# Core imports
# ==========================
import os
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from PIL import Image
import clip
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

import polars as pl
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryAUROC
from tqdm import tqdm


warnings.filterwarnings("ignore")

2025-12-18 14:28:33.926869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766068114.321581      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766068114.450550      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# 2. Dataset Overview

We use the MicroLens-1M dataset, which contains user click interactions and rich item content features such as titles and cover images.

The dataset is split into training, validation, and test sets. The training and
validation sets are used for model optimization and hyperparameter tuning, while
the test set is reserved for final inference and submission generation.


In [4]:
# ==========================
# Dataset paths (Kaggle)
# ==========================
INFO_FILE = "/kaggle/input/item_info.parquet"
FEATURE_FILE = "/kaggle/input/item_feature.parquet"
IMAGE_DIR = "/kaggle/input/item_images/item_images/"

SAVE_PATH = "/kaggle/working/item_info_multimodal_emb.parquet"

# ==========================
# Read parquet files
# ==========================
df_info = pd.read_parquet(INFO_FILE)
df_features = pd.read_parquet(FEATURE_FILE)

print("Item info shape :", df_info.shape)
print("Item feature shape :", df_features.shape)


Item info shape : (91718, 3)
Item feature shape : (91717, 7)


# 3. Task 1: Multimodal Item Embedding
In this section, we extract textual and visual representations for each item, then fuse them into a compact 128-dimensional embedding.


# 3.1 Text Embedding and Image Embedding

We encode item textual information using a pretrained sentence-level encoder to obtain dense semantic representations: Sentence-BERT

Item cover images are processed using a pretrained vision model to extract visual embeddings: CLIP-ResNet50

In [5]:
# ==========================
# Device configuration
# ==========================
compute_device = "cuda" if torch.cuda.is_available() else "cpu"

# ==========================
# Text encoder (Sentence-BERT)
# ==========================
text_encoder = SentenceTransformer("all-MiniLM-L6-v2")
text_encoder.eval()

# ==========================
# Image encoder (CLIP ResNet-50)
# ==========================
clip_net, clip_preprocess = clip.load("RN50", device=compute_device)
clip_net.eval()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|████████████████████████████████████████| 244M/244M [00:01<00:00, 130MiB/s]


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
     

In [6]:
# ==========================
# Text embeddings
# ==========================
item_titles = df_features["item_title"].tolist()

text_vectors = text_encoder.encode(
    item_titles,
    batch_size=256,
    show_progress_bar=True
)

# ==========================
# Image embeddings 
# ==========================
def extract_image_embedding(image_path):
    """
    Compute CLIP embedding for a single image.
    If the image is missing or corrupted, return a zero vector.
    """
    try:
        img = Image.open(image_path).convert("RGB")
        img_tensor = clip_preprocess(img).unsqueeze(0).to(compute_device)

        with torch.no_grad():
            img_emb = clip_net.encode_image(img_tensor)

        return img_emb.cpu().numpy().squeeze()

    except Exception:
        return np.zeros(clip_net.visual.output_dim)


# ==========================
# Encode all item images
# ==========================
image_vectors = []

for iid in df_features["item_id"]:
    img_path = os.path.join(IMAGE_DIR, f"image{iid}.jpg")
    image_vectors.append(extract_image_embedding(img_path))

image_vectors = np.array(image_vectors)

print("Image embedding matrix shape:", image_vectors.shape)


Batches:   0%|          | 0/359 [00:00<?, ?it/s]

Image embedding matrix shape: (91717, 1024)


In [7]:
# Inspect available columns
print(df_info.columns.tolist())

['item_id', 'item_tags', 'item_emb_d128']


# 3.3 Multimodal Fusion and Dimensionality Reduction

Text and image embeddings are concatenated to form multimodal representations. To satisfy competition constraints and reduce inference latency, PCA is applied to project embeddings to 128 dimensions.

In [8]:
from IPython.display import FileLink

# ==========================
# Multimodal fusion (text + image)
# ==========================
multimodal_features = np.concatenate(
    [text_vectors, image_vectors],
    axis=1
)

print("Fused embedding shape:", multimodal_features.shape)

# ==========================
# Padding first row (alignment with item_info)
# ==========================
zero_padding = np.zeros((1, multimodal_features.shape[1]))
multimodal_features_padded = np.vstack(
    [zero_padding, multimodal_features]
)

print("After padding:", multimodal_features_padded.shape)

# ==========================
# Dimensionality reduction (PCA → 128)
# ==========================
pca_model = PCA(n_components=128)
multimodal_128d = pca_model.fit_transform(multimodal_features_padded)

print("Final embedding shape:", multimodal_128d.shape)

# ==========================
# Save into item_info
# ==========================
df_info["item_emb_d128"] = list(multimodal_128d)
df_info.to_parquet(SAVE_PATH, index=False)
#You can download the saved file from /kaggle/working/ directory
print(f"Saved fused embeddings to: {SAVE_PATH}")

Fused embedding shape: (91717, 1408)
After padding: (91718, 1408)
Final embedding shape: (91718, 128)
Saved fused embeddings to: /kaggle/working/item_info_multimodal_emb.parquet


In [9]:
# Reload saved file for sanity check
df_check = pd.read_parquet(SAVE_PATH)

print(df_check.head())
print("Embedding dimension example:", len(df_check["item_emb_d128"].iloc[0]))

   item_id        item_tags                                      item_emb_d128
0        0  [0, 0, 0, 0, 0]  [-0.09303953632827579, -0.023277998817579422, ...
1        1  [0, 0, 0, 0, 1]  [-0.012815042967732978, -0.09739979676869429, ...
2        2  [0, 0, 2, 3, 4]  [-0.04312385184114877, 0.007002070534522362, -...
3        3  [0, 0, 5, 6, 7]  [0.11733471759868354, -0.07496687006709006, -0...
4        4  [0, 0, 0, 8, 9]  [0.02941412000516612, -0.02858132508637742, -0...
Embedding dimension example: 128


# 4. Task 2: Multimodal CTR Prediction

In this task, we focus on predicting the click-through rate (CTR) by effectively leveraging the multimodal item embeddings generated in Task 1. The objective is to design an end-to-end neural CTR model that integrates user interaction features with multimodal item representations.

The model takes as input user identifiers, target items, historical interaction sequences, and engagement signals (likes and views), together with the 128-dimensional multimodal item embeddings. These features are jointly learned through embedding layers and a multilayer perceptron (MLP) to estimate the probability of a user clicking on a given item.

The model is trained using binary cross-entropy loss and evaluated using the Area Under the ROC Curve (AUC), following the official data split and competition constraints.


# 4.1 Problem Formulation

Given user interaction history and item multimodal embeddings, the task is to predict the probability of a click (pCTR; I named it Task1&2 in the submission.csv as you'll find at the end of this notebook) for each user–item pair.

In [10]:
# ==========================
# Kaggle dataset paths
# ==========================
BASE_PATH = "/kaggle/input/"  

train_df = pl.read_parquet(f"{BASE_PATH}/train.parquet")
valid_df = pl.read_parquet(f"{BASE_PATH}/valid.parquet")
test_df  = pl.read_parquet(f"{BASE_PATH}/test.parquet")

# Load item_info WITH multimodal embeddings
item_info = pl.read_parquet(
    "/kaggle/working/item_info_multimodal_emb.parquet"
)

print(
    "Train / Valid / Test shapes:",
    train_df.shape, valid_df.shape, test_df.shape
)

Train / Valid / Test shapes: (3600000, 6) (10000, 6) (379142, 6)


In [11]:
# ==========================
# Device configuration
# ==========================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


### Feature Engineering

In addition to raw identifiers, we engineer auxiliary features to improve model
generalization. In particular, item popularity is computed from the training
data and log-scaled to reduce skewness. This feature captures global item
exposure patterns and complements personalized user representations.


In [13]:
# ==========================
# Item Embeddings + Feature Engineering
# ==========================
item_emb_matrix = np.stack(
    item_info["item_emb_d128"].to_numpy()
)

item_emb_tensor = torch.tensor(
    item_emb_matrix,
    dtype=torch.float32,
    device=device
)

num_items = item_emb_tensor.shape[0]
print("Number of items (including padding):", num_items)

# Compute item popularity for better features
item_popularity = train_df.group_by("item_id").agg(
    pl.count("user_id").alias("item_count")
).sort("item_id")

# Create popularity tensor (log-scaled)
item_pop_dict = dict(zip(
    item_popularity["item_id"].to_list(),
    item_popularity["item_count"].to_list()
))

item_pop_array = np.array([
    np.log1p(item_pop_dict.get(i, 0)) for i in range(num_items)
])

item_popularity_tensor = torch.tensor(
    item_pop_array,
    dtype=torch.float32,
    device=device
)

print("Item popularity features computed")

Number of items (including padding): 91718
Item popularity features computed


# 4.2 Model Architecture

We design an end-to-end neural CTR model that ingests user features and item multimodal embeddings. The model outputs a scalar pCTR via fully connected layers.

The custom dataset class handles data preparation for both training and
inference. It supports variable-length interaction sequences and explicitly
computes sequence length as an additional numerical feature.

In [14]:
# ==========================
# CTR Dataset
# ==========================
class CTRDataset(Dataset):

    def __init__(self, df, is_test=False):
        self.is_test = is_test

        self.user_id  = df["user_id"].to_numpy()
        self.item_id  = df["item_id"].to_numpy()
        self.item_seq = df["item_seq"].to_numpy()
        self.likes    = df["likes_level"].to_numpy()
        self.views    = df["views_level"].to_numpy()

        # Compute sequence length as feature
        self.seq_len = np.array([
            np.count_nonzero(seq) for seq in self.item_seq
        ])

        if not is_test:
            self.label = df["label"].to_numpy()
        else:
            self.ID = df["ID"].to_numpy()

    def __len__(self):
        return len(self.user_id)

    def __getitem__(self, idx):
        sample = {
            "user_id": torch.tensor(self.user_id[idx], dtype=torch.long),
            "item_id": torch.tensor(self.item_id[idx], dtype=torch.long),
            "item_seq": torch.tensor(self.item_seq[idx], dtype=torch.long),
            "likes": torch.tensor(self.likes[idx], dtype=torch.long),
            "views": torch.tensor(self.views[idx], dtype=torch.long),
            "seq_len": torch.tensor(self.seq_len[idx], dtype=torch.float32),  # 🆕 NEW
        }

        if self.is_test:
            sample["ID"] = torch.tensor(self.ID[idx], dtype=torch.long)
        else:
            sample["label"] = torch.tensor(self.label[idx], dtype=torch.float32)

        return sample

In [15]:
# ==========================
# DataLoaders 
# ==========================
BATCH_SIZE = 1024

train_loader = DataLoader(
    CTRDataset(train_df),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,      # IMPORTANT for Kaggle users
    pin_memory=False    # IMPORTANT for Kaggle users
)

valid_loader = DataLoader(
    CTRDataset(valid_df),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False
)

test_loader = DataLoader(
    CTRDataset(test_df, is_test=True),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False
)


## Model Architecture

We propose an enhanced multimodal CTR prediction model that combines embedding
layers, attention-based sequence modeling, feature interaction mechanisms, and a
deep multilayer perceptron (MLP).


Multi-head attention is used to aggregate sequential item interactions, enabling
the model to focus on the most relevant past behaviors. Multimodal item
embeddings are projected into the same latent space as categorical features to
allow effective fusion.

In [16]:
# ==========================
# ENHANCED MMCTR Model
# ==========================
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_dim, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = emb_dim // num_heads
        
        self.qkv = nn.Linear(emb_dim, emb_dim * 3)
        self.out = nn.Linear(emb_dim, emb_dim)
        
    def forward(self, x, mask=None):
        B, L, D = x.shape
        
        qkv = self.qkv(x).reshape(B, L, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = (q @ k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        if mask is not None:
            attn = attn.masked_fill(mask.unsqueeze(1).unsqueeze(2) == 0, -1e9)
        
        attn = F.softmax(attn, dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(B, L, D)
        
        return self.out(out)


class EnhancedMMCTRModel(nn.Module):
    def __init__(self, num_users, num_items, emb_dim=64):  
        super().__init__()

        # Embeddings (larger capacity)
        self.user_emb = nn.Embedding(num_users + 1, emb_dim, padding_idx=0)
        self.item_emb = nn.Embedding(num_items, emb_dim, padding_idx=0)
        self.likes_emb = nn.Embedding(11, emb_dim)
        self.views_emb = nn.Embedding(11, emb_dim)

        # Multi-head attention for sequence
        self.attention = MultiHeadAttention(emb_dim, num_heads=4)
        
        # Transform multimodal embeddings
        self.mm_transform = nn.Sequential(
            nn.Linear(128, emb_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        # Feature interaction layer (FM-style)
        self.feature_interaction = nn.Bilinear(emb_dim, emb_dim, emb_dim)

        # Deeper MLP with residual connections and BatchNorm
        self.bn1 = nn.BatchNorm1d(emb_dim * 6 + 1)  # +1 for seq_len
        
        self.mlp1 = nn.Sequential(
            nn.Linear(emb_dim * 6 + 1, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.mlp2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.mlp3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        self.output = nn.Linear(128, 1)

        # Initialize weights properly
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, mean=0, std=0.01)

    def forward(self, user_id, item_id, item_seq, likes, views, seq_len):
        # Basic embeddings
        user_e = self.user_emb(user_id)
        item_e = self.item_emb(item_id)
        likes_e = self.likes_emb(likes)
        views_e = self.views_emb(views)

        # Attention-based sequence pooling
        seq_e = self.item_emb(item_seq)
        mask = (item_seq != 0)
        
        # Apply attention
        seq_attended = self.attention(seq_e, mask)
        
        # Pool with attention weights
        seq_pooled = (seq_attended * mask.unsqueeze(-1)).sum(1) / (mask.sum(1, keepdim=True) + 1e-8)

        # Multimodal item embedding (transformed)
        item_mm_e = item_emb_tensor[item_id]
        item_mm_e = self.mm_transform(item_mm_e)

        # Feature interaction (user-item, item-sequence)
        ui_interaction = self.feature_interaction(user_e, item_e)

        # Normalize sequence length
        seq_len_norm = (seq_len.unsqueeze(1) / 50.0).clamp(0, 1)

        # Concatenate all features
        x = torch.cat(
            [user_e, item_e, seq_pooled, likes_e, views_e, item_mm_e, seq_len_norm],
            dim=1
        )

        # BatchNorm before MLP
        x = self.bn1(x)

        # Deep MLP with residual-style connections
        x1 = self.mlp1(x)
        x2 = self.mlp2(x1)
        x3 = self.mlp3(x2)

        return self.output(x3).squeeze(1)


In [17]:
# ==========================
#  Model Setup
# ==========================
num_users = int(train_df["user_id"].max())

model = EnhancedMMCTRModel(
    num_users=num_users,
    num_items=num_items,
    emb_dim=64  
).to(device)

# Label smoothing for better generalization
class BCEWithLogitsLossSmoothed(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.bce = nn.BCEWithLogitsLoss()
    
    def forward(self, logits, targets):
        targets = targets * (1 - self.smoothing) + 0.5 * self.smoothing
        return self.bce(logits, targets)

criterion = BCEWithLogitsLossSmoothed(smoothing=0.05)

# AdamW with weight decay for better regularization
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-5
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=2,
    verbose=True
)

auc_metric = BinaryAUROC().to(device)

print(f"✅ Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

✅ Model initialized with 70,523,075 parameters


# 4.3 Model Training and Validation

The multimodal CTR model is trained on the training set and evaluated on the validation set using the Area Under the ROC Curve (AUC), a standard metric for imbalanced binary classification. Training uses binary cross-entropy loss with label smoothing to improve generalization and the AdamW optimizer with weight decay to mitigate overfitting. Gradient clipping is applied to stabilize optimization, while early stopping based on validation AUC ensures optimal model selection for final inference.


In [18]:
# ==========================
# Training Loop
# ==========================
EPOCHS = 30  
best_valid_auc = 0.0
patience = 5  
patience_counter = 0

MODEL_PATH = "/kaggle/working/best_mmctr_model.pt"

for epoch in range(EPOCHS):

    # ===== TRAIN =====
    model.train()
    epoch_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()

        logits = model(
            batch["user_id"].to(device),
            batch["item_id"].to(device),
            batch["item_seq"].to(device),
            batch["likes"].to(device),
            batch["views"].to(device),
            batch["seq_len"].to(device) 
        )

        loss = criterion(logits, batch["label"].to(device))
        loss.backward()
        
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()

        epoch_loss += loss.item()

    avg_train_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}")

    # ===== VALID =====
    model.eval()
    auc_metric.reset()

    with torch.no_grad():
        for batch in valid_loader:
            logits = model(
                batch["user_id"].to(device),
                batch["item_id"].to(device),
                batch["item_seq"].to(device),
                batch["likes"].to(device),
                batch["views"].to(device),
                batch["seq_len"].to(device)  
            )
            probs = torch.sigmoid(logits)
            auc_metric.update(probs, batch["label"].to(device))

    valid_auc = auc_metric.compute().item()
    print(f"Epoch {epoch+1} | Valid AUC: {valid_auc:.4f}")

    # Learning rate scheduling
    scheduler.step(valid_auc)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Current LR: {current_lr:.6f}")

    # ===== EARLY STOPPING =====
    if valid_auc > best_valid_auc:
        best_valid_auc = valid_auc
        patience_counter = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"🔥 New best model saved (AUC = {best_valid_auc:.4f})")
    else:
        patience_counter += 1
        print(f"⏳ No improvement ({patience_counter}/{patience})")

    if patience_counter >= patience:
        print("🛑 Early stopping triggered")
        break

print(f"\n✅ Training complete! Best Valid AUC: {best_valid_auc:.4f}")

Epoch 1: 100%|██████████| 3516/3516 [05:56<00:00,  9.86it/s]


Epoch 1 | Train Loss: 0.2221
Epoch 1 | Valid AUC: 0.9135
Current LR: 0.001000
🔥 New best model saved (AUC = 0.9135)


Epoch 2: 100%|██████████| 3516/3516 [05:54<00:00,  9.93it/s]


Epoch 2 | Train Loss: 0.1500
Epoch 2 | Valid AUC: 0.9146
Current LR: 0.001000
🔥 New best model saved (AUC = 0.9146)


Epoch 3: 100%|██████████| 3516/3516 [05:55<00:00,  9.88it/s]


...

Epoch 5 | Train Loss: 0.1230
Epoch 5 | Valid AUC: 0.8781
Current LR: 0.000500
⏳ No improvement (3/5)


Epoch 6: 100%|██████████| 3516/3516 [05:57<00:00,  9.84it/s]


Epoch 6 | Train Loss: 0.1200
Epoch 6 | Valid AUC: 0.8994
Current LR: 0.000500
⏳ No improvement (4/5)


Epoch 7: 100%|██████████| 3516/3516 [06:07<00:00,  9.57it/s]


Epoch 7 | Train Loss: 0.1184
Epoch 7 | Valid AUC: 0.8858
Current LR: 0.000500
⏳ No improvement (5/5)
🛑 Early stopping triggered

✅ Training complete! Best Valid AUC: 0.9146


# 5. Test Inference and Submission

During inference, the best-performing model checkpoint is loaded and applied to
the test set. Predicted probabilities are exported in the required submission
format.


In [19]:
# ==========================
# Load best trained model
# ==========================
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

test_ids = []
test_pctr = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Inference on test"):
        logits = model(
            batch["user_id"].to(device),
            batch["item_id"].to(device),
            batch["item_seq"].to(device),
            batch["likes"].to(device),
            batch["views"].to(device),
            batch["seq_len"].to(device)   
        )

        probs = torch.sigmoid(logits)

        test_ids.extend(batch["ID"].cpu().numpy())
        test_pctr.extend(probs.cpu().numpy())

Inference on test: 100%|██████████| 371/371 [00:23<00:00, 15.74it/s]


In [20]:
# ==========================
# Build submission file
# ==========================
import pandas as pd

submission_df = pd.DataFrame({
    "ID": test_ids,
    "Task1&2": test_pctr
})

SUB_PATH = "/kaggle/working/submission.csv"
submission_df.to_csv(SUB_PATH, index=False)

print("Submission file saved at:", SUB_PATH)


Submission file saved at: /kaggle/working/submission.csv


In [21]:
assert len(submission_df) == test_df.shape[0]
assert submission_df["Task1&2"].between(0, 1).all()
assert not submission_df.isnull().any().any()
assert submission_df["ID"].is_unique

print("✅ Submission sanity check passed")

✅ Submission sanity check passed


## 7. Conclusion
We implemented a complete multimodal CTR prediction pipeline, combining textual and visual item information with user interaction data. The proposed model improves CTR prediction performance and satisfies the low-latency and reproducibility constraints of the MM-CTR Challenge.